In [15]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint
from IPython.display import display
from tqdm import tqdm
import glob
# import mlflow 
from langsmith import Client

%load_ext autoreload
%autoreload 2
%matplotlib inline


sys.path.append(os.path.dirname(os.getcwd()))
from src import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
# use the already-imported src.utils (imported in an earlier cell)
from src.config import load_config

config = load_config()

In [18]:
from src.runner.state import PlannerStateMaker


state_maker = PlannerStateMaker()

state = state_maker.make(user_query="get a apple", context="")

In [19]:
from src.runner.runner import PlanRunner

runner = PlanRunner(config=config, result_key="actions")

In [20]:
pprint(config.skills)

[RobotSkillConfig(name='robot1', skills=['GoToObject', 'OpenObject', 'CloseObject', 'PickObject', 'PlaceObject']),
 RobotSkillConfig(name='robot2', skills=['GoToObject', 'OpenObject', 'CloseObject', 'PickObject', 'PlaceObject'])]


In [28]:
from src.config.config import RobotSkillConfig


def make_skills_text(config_skills: list[RobotSkillConfig]) -> str:
    skills_text_list = []
    for robot_skill in config_skills:

        skills_text = f"from {robot_skill.name}.skills import "
        for skill in robot_skill.skills:
            skills_text += f"{skill}"
            if skill != robot_skill.skills[-1]:
                skills_text += ", "
        skills_text_list.append(skills_text)

    return "\n".join(skills_text_list)


skills_text = make_skills_text(config.skills)
print(skills_text)

from robot1.skills import GoToObject, OpenObject, CloseObject, PickObject, PlaceObject
from robot2.skills import GoToObject, OpenObject, CloseObject, PickObject, PlaceObject


In [36]:
# object_map = utils.load("./data/object_map.pkl")
# pprint(list(object_map.keys()))

In [29]:
def make_objects_text(objects_env):
    pass


objects_text = make_objects_text(None)